In [1]:
import tensorflow as tf
from siren import Siren
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from tqdm import trange
from Adam import Adam

2023-11-25 18:34:57.150706: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-25 18:34:58.330052: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-25 18:34:58.345548: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-25 18

In [2]:
def get_image(image_path):
    img = np.asarray(Image.open(image_path))
    return img
img = get_image('Testcard_F.jpg')/255
img = img.astype(np.float32)
#moments = tf.nn.moments(img, axes=[0, 1])
#moments
img.shape

(273, 365, 3)

In [3]:
def get_loss(true, guess):
    tf.reduce_mean()

In [4]:
test = img.reshape(-1, 3)
test[1*len(img[0]) + 2]

array([0.99607843, 0.99607843, 0.9882353 ], dtype=float32)

In [5]:
tf.gather_nd(img, [[0,1], [2,3]])

grid_shape = (273, 365)

# Generate meshgrid
x_vals, y_vals = tf.meshgrid(tf.range(273), tf.range(365))
x_vals
# Reshape to create pairs of (x, y) coordinates
coordinates = tf.stack([ tf.reshape(y_vals, -1), tf.reshape(x_vals, -1)], axis=1)
coordinates[99000]


<tf.Tensor: shape=(2,), dtype=int32, numpy=array([362, 174], dtype=int32)>

In [6]:

siren_model = Siren(num_inputs=2, num_outputs=3, num_hidden_layers=5, hidden_layer_width=256)
x_values = tf.random.uniform(shape=[512], maxval=273, dtype=tf.int32)
y_values = tf.random.uniform(shape=[512], maxval=365, dtype=tf.int32)
xy_values = tf.stack([x_values, y_values], axis=1)
image_val = tf.gather_nd(img, xy_values)
out = siren_model(tf.cast(xy_values, dtype=tf.float32))
out = siren_model(tf.cast(coordinates, dtype=tf.float32))
test = tf.reshape(out,shape=[273, 365, -1])
print(out[1*len(img[0]) + 2])
print(test[1][2])

tf.Tensor([ 0.5914596  -0.781914    0.99795276], shape=(3,), dtype=float32)
tf.Tensor([ 0.5914596  -0.781914    0.99795276], shape=(3,), dtype=float32)


2023-11-25 18:34:58.827158: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [7]:
iterations =  5000
bar = trange(iterations)
optimizer = Adam(1e-4)
x_vals, y_vals = tf.meshgrid(tf.range(365), tf.range(273))

# Reshape to create pairs of (x, y) coordinates
cord_vals = tf.stack([ tf.reshape(y_vals, -1), tf.reshape(x_vals, -1)], axis=1)
image_true = tf.gather_nd(img, cord_vals)
for i in bar:
    with tf.GradientTape() as tape:
      #  x_values = tf.random.uniform(shape=[5000], maxval=273, dtype=tf.int32)
       # y_values = tf.random.uniform(shape=[5000], maxval=365, dtype=tf.int32)
        
        xy_values = tf.stack([x_values, y_values], axis=1)
        
        model_output = siren_model(tf.cast(cord_vals, dtype=tf.float32))
        loss = tf.reduce_mean((model_output - image_true)**2)

        grads = tape.gradient(loss, siren_model.trainable_variables)
        optimizer.apply_gradients(grads, siren_model.trainable_variables)

        if i % 10 == (10 - 1):
            bar.set_description(f"Step {i}; loss => {loss.numpy():0.4f}")
            bar.refresh()
        
out = tf.reshape(model_output,shape=[273, 365, -1])
plt.imshow(out)

Step 849; loss => 0.0022:  85%|████████▌ | 851/1000 [00:37<00:06, 22.26it/s]